In [184]:
!pip install optuna -q

In [194]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

import optuna

import sklearn
from sklearn import linear_model as lm

In [195]:

X_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0)
X_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0)

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0)
y['Bound'] = y.Bound.apply(lambda x: -1 if x == 0 else 1)
y=y.values
# y.head()

In [196]:
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def compare(beta1, beta2):
    print('''
Difference between the two:
{}
        '''.format(np.sum((beta1-beta2)**2))
    )

In [197]:
n = 6


X_te = []
X = []
cv = CountVectorizer()
for i in X_train:
    sentence = ' '.join(getKmers(i[0], size=n))
    X.append(sentence)
    
for i in X_test:
    sentence = ' '.join(getKmers(i[0], size=n))
    X_te.append(sentence)

    
X_later = X+X_te
X = cv.fit_transform(X_later).toarray()
# breakS
    

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
X[:15,]

In [157]:
print('x_train: {} y_train {}'.format(X[:2000,:].shape,y.shape))

# print('x_train: {} y_train {}'.format(X_train.shape,y.shape))
# print('x_test: {}'.format(X_te.shape))

x_train: (2000, 150) y_train (2000, 1)


In [158]:
X_tr= scale(X[:2000,:])
X_te = scale(X[2000:,:])

X_train, X_val, y_train, y_val = train_test_split(
    X_tr, y, test_size=0.3, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape,X_test.shape)

(1400, 150) (600, 150) (1400, 1) (600, 1) (1000, 1)


### Model Test

In [159]:
# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam * n
    b = X.T.dot(y)
    
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

# Weighted Ridge Regression (WRR)
def solveWRR(y, X, w, lam):
    n, p = X.shape
    assert (len(y) == len(w) == n)
    
    y1 = np.sqrt(w) * y
    X1 = (np.sqrt(w) * X.T).T
    # Hint:
    # Find y1 and X1 such that:
    beta = solveRR(y1, X1, lam)
    return (beta)

In [160]:
len(y_train)

1400

In [161]:
lam = 10

w = np.random.rand(len(y_train))
# Our solver
beta1 = solveWRR(y_train, X_train,w, lam)

# Python solver
alpha = lam * X_train.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X_train,y_train).coef_

# Check
# compare(beta1, beta2)
beta2.shape

(1, 150)

# Ridge Regression (RR)

In [162]:
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] = np.add(A[np.diag_indices_from(A)], lam * n)
    b = X.T.dot(y)
    
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

In [163]:
# def cross_validate(x_data,y_data,lr,lamda=0.2,epoch=10,k=5,decay=10):
#     if len(x_data)%k != 0:
#         print('cant vsplit',len(x_data),' by ',k)
#         return
    
#     x_data_splitted = np.vsplit(x_data,k)
#     y_data_splitted = np.vsplit(y_data,k)
    
#     aggrigate_result = []
#     for i in range(len(x_data_splitted)):
#         train = []
#         test = []
#         items = [j for j in range(len(x_data_splitted)) if j !=i ]
#         x_test = x_data_splitted[i]
#         y_test = y_data_splitted[i]
#         for item in items:
#             if len(train) == 0:
#                 x_train = x_data_splitted[item]
#                 y_train = y_data_splitted[item]
#             else:
#                 x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
#                 y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
#         logistic = logisticregression(x_train,y_train,lamda=lamda,lr=lr,decay=decay,epoch=epoch)
#         logistic.train()
        
#         result = logistic.evaluate(x_test,y_test)
#         aggrigate_result.append(result)
        
#         value = sum(aggrigate_result)/len(aggrigate_result)
#     return value if value!= None else 0

In [164]:
# cross_validate(X_tr, y,lam,5)

In [165]:
def objective(trial):
    lam = trial.suggest_loguniform('lamda', 0.01, 5)

    # Our solver
    beta1 = solveRR(y_train, X_train, lam)

    # Python solver
    alpha = lam *  X_train.shape[0]
    model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
    beta2 = model.fit(X_train,y_train).coef_
    
    valid_acc = sum(np.sign(np.dot(X_val,beta1))==y_val)/len(y_val)
    train_acc = sum(np.sign(np.dot(X_train,beta1))==y_train)/len(y_train)
    
    return valid_acc

In [166]:

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=50,show_progress_bar=False)

[I 2020-05-26 15:32:34,490] Finished trial#0 with value: 0.6066666666666667 with parameters: {'lamda': 0.5256211664635327}. Best is trial#0 with value: 0.6066666666666667.
[I 2020-05-26 15:32:34,606] Finished trial#1 with value: 0.6133333333333333 with parameters: {'lamda': 0.1260265087281257}. Best is trial#1 with value: 0.6133333333333333.
[I 2020-05-26 15:32:34,704] Finished trial#2 with value: 0.6216666666666667 with parameters: {'lamda': 0.033431459825172954}. Best is trial#2 with value: 0.6216666666666667.
[I 2020-05-26 15:32:34,807] Finished trial#3 with value: 0.6016666666666667 with parameters: {'lamda': 1.5297769496966853}. Best is trial#2 with value: 0.6216666666666667.
[I 2020-05-26 15:32:34,914] Finished trial#4 with value: 0.605 with parameters: {'lamda': 0.7766828789687985}. Best is trial#2 with value: 0.6216666666666667.
[I 2020-05-26 15:32:35,050] Finished trial#5 with value: 0.6033333333333334 with parameters: {'lamda': 0.374354481602087}. Best is trial#2 with value: 

In [167]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)

df.sort_values(by=['value'])

,number,value,duration,params_lamda
25,25,0.595000,00:00:00.129606,4.595150
36,36,0.600000,00:00:00.112667,0.867843
3,3,0.601667,00:00:00.099266,1.529777
5,5,0.603333,00:00:00.132510,0.374354
8,8,0.605000,00:00:00.113079,0.360092
4,4,0.605000,00:00:00.101604,0.776683
0,0,0.606667,00:00:00.125164,0.525621
9,9,0.608333,00:00:00.104613,0.280644
40,40,0.611667,00:00:00.097310,0.207472
35,35,0.613333,00:00:00.108289,0.107214


In [15]:
beta1 = solveRR(y_train, X_train, 4.962821)

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

In [194]:
np.sum(np.sign(np.dot(X_train,beta1))==y_train)/len(y_train)

0.9544444444444444

In [195]:
np.sum(np.sign(np.dot(X_val,beta1))==y_val)/len(y_val)

0.66

In [196]:
sol = []

for i in range(len(X_te)):
    result = np.sign(np.dot(X_te[i],beta1))
    if result == 1:
        sol.append([i,1])
    else:
        sol.append([i,0])
                     

In [197]:
d = pd.DataFrame(sol)
d.columns = ['Id','Bound']
d.head()

,Id,Bound
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


In [198]:
d.to_csv('test_65_rr_solver.csv',index=False)

# Kernel Trick

In [216]:
# def polynomial_kernel(x, y, p=2):
#     return (1 + np.dot(x, y)) ** p

In [225]:
# def solveRR(y, X, lam):
#     n, p = X.shape
#     assert (len(y) == n)
    
    
    
#     A = X.T.dot(X)
#     # Adjust diagonal due to Ridge
#     A[np.diag_indices_from(A)] += lam * n
#     b = X.T.dot(y)
    
#     # Hint:
#     beta = np.linalg.solve(A, b)
#     # Finds solution to the linear system Ax = b
#     return (beta)

In [227]:
# beta1 = solveRR(y_train, X_train, 2)
# beta1.shape

In [228]:
# def objective(trial):
#     lam = trial.suggest_loguniform('lam', 0.01, 5)

#     # Our solver
#     beta1 = solveRR(y_train, X_train, lam)

#     # Python solver
#     alpha = lam *  X_train.shape[0]
#     model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
#     beta2 = model.fit(X_train,y_train).coef_
    
#     valid_acc = sum(np.sign(np.dot(X_val,beta1))==y_val)/len(y_val)
#     train_acc = sum(np.sign(np.dot(X_train,beta1))==y_train)/len(y_train)
    
#     return valid_acc

In [229]:

# import optuna

# sampler = optuna.samplers.TPESampler()
# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective, n_trials=50,show_progress_bar=False)

# WRR

In [199]:
# Weighted Ridge Regression (WRR)
def solveWRR(y, X, w, lam):
    n, p = X.shape
    assert (len(y) == len(w) == n)
    
    y1 = np.sqrt(w) * y
    X1 = (np.sqrt(w) * X.T).T
    # Hint:
    # Find y1 and X1 such that:
    beta = solveRR(y1, X1, lam)
    return (beta)

In [201]:
lam = 0.1
w = np.random.rand(len(y_train))

# Our solver
beta1 = solveWRR(y_train, X_train, w, lam)

# Python solver
alpha = lam * X_train.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X_train, y_train, sample_weight=w).coef_

# Check
# compare(beta1, beta2)
beta1.shape,beta2.shape

((4096, 1800), (1, 4096))